In [1]:
import numpy as np
import cv2

In [2]:
mtx = np.array([[550.742541314096,   0,         309.3349978427164],
                [  0,         551.2202584096859, 226.39975185622637],
                [  0,           0,           1        ]])

dist = np.array([0.15262652,  0.33736018,  0.01009325,  0.00782211, -1.21224862])


In [3]:
def basis_from_deg(camera_degrees):

    x, y, z = np.deg2rad(camera_degrees)

    rx = np.array([[1, 0, 0],
                   [0, np.cos(x), -np.sin(x)],
                   [0, np.sin(x), np.cos(x)]])
    
    ry = np.array([[np.cos(y), 0, np.sin(y)],
                   [0, 1, 0],
                   [-np.sin(y), 0, np.cos(y)]])

    rz = np.array([[np.cos(z), -np.sin(z), 0],
                   [np.sin(z), np.cos(z), 0],
                   [0, 0, 1]])
    
    basis = np.matmul(rz, np.matmul(ry, rx))
    
    return basis

In [4]:
imageSize = (640, 480)

# Rotation matrices for both cameras
R1 = basis_from_deg([0, 0, 0])  # Replace with your left camera rotation matrix
R2 = R1.copy()  # Replace with your right camera rotation matrix

# Translation vector between the cameras
T = np.array([8.2, 0 ,0])  # Replace with your translation vector

# Calculate relative rotation
R_relative = np.dot(R2, R1.T)

# Perform stereo rectification
R1_rect, R2_rect, P1, P2, Q, _, _ = cv2.stereoRectify(
    mtx, dist,
    mtx, dist,
    imageSize, R_relative, T
)

# After this, you can proceed with creating the rectification maps and remapping the images


In [5]:
print(R1_rect, R2_rect)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]] [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [6]:
print(P1, P2)

[[551.22025841   0.         311.76721954   0.        ]
 [  0.         551.22025841 229.44722748   0.        ]
 [  0.           0.           1.           0.        ]] [[5.51220258e+02 0.00000000e+00 3.11767220e+02 4.52000612e+03]
 [0.00000000e+00 5.51220258e+02 2.29447227e+02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00]]


In [7]:
print(R_relative)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [8]:
import cv2
import numpy as np


cameraMatrix1 = mtx  
distCoeffs1 = np.zeros((1, 5))    
distCoeffs2 = np.zeros((1, 5))
R = R_relative             
T = T      

imageSize = (1152, 648)

# rectification
R1, R2, P1, P2, Q, _, _ = cv2.stereoRectify(
    cameraMatrix1, distCoeffs1, 
    cameraMatrix2, distCoeffs2, 
    imageSize, R, T, 
    alpha=0
)

# rectification map
map1x, map1y = cv2.initUndistortRectifyMap(
    cameraMatrix1, distCoeffs1, R1, P1, imageSize, cv2.CV_32FC1
)
map2x, map2y = cv2.initUndistortRectifyMap(
    cameraMatrix2, distCoeffs2, R2, P2, imageSize, cv2.CV_32FC1
)

imgL = cv2.imread("Images/tsukuba_left.png")
imgR = cv2.imread("Images/tsukuba_right.png")

#rectifying
rectifiedL = cv2.remap(imgL, map1x, map1y, cv2.INTER_LINEAR)
rectifiedR = cv2.remap(imgR, map2x, map2y, cv2.INTER_LINEAR)

cv2.imwrite('rectified_left.jpg', rectifiedL)
cv2.imwrite('rectified_right.jpg', rectifiedR)


while True:
    cv2.imshow('Left Image', imgL)
    cv2.imshow('Right Image', imgR)
    cv2.imshow('Rectified Left Image', rectifiedL)
    cv2.imshow('Rectified Right Image', rectifiedR)
    if cv2.waitKey(1) & 0xFF == 27:  # Press 'Esc' to exit
        break

cv2.destroyAllWindows()


cv2.waitKey(0)
cv2.destroyAllWindows()


NameError: name 'cameraMatrix2' is not defined

In [9]:
def depth_from_disparity(disparity):
    return (1 * mtx[0][0]) / disparity

In [10]:
def depth_f(b, fx, u1, u2):
    return (b * fx) / (u1 - u2)

In [11]:
print(depth_f(0.1, 420, 544, 522))

1.9090909090909092


In [12]:
stereo = cv2.StereoBM.create(numDisparities=48, blockSize=9)

left = cv2.cvtColor(rectifiedL, cv2.COLOR_BGR2GRAY)
right = cv2.cvtColor(rectifiedR, cv2.COLOR_BGR2GRAY)

disparity = stereo.compute(right, left)
while True:
    # cv2.imshow('left', left)
    # cv2.imshow('right', right)
    # cv2.imshow('disparity', disparity)
    depth_image = depth_from_disparity(disparity)
    cv2.imshow('depth', depth_image)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()

NameError: name 'rectifiedL' is not defined